# LightGBM の実装

In [1]:
# ====================================================
# Library
# ====================================================
import os
import gc
import datetime
import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np
import pandas as pd
from pathlib import Path
import pickle

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

In [2]:
!mkdir oof
!mkdir models

mkdir: oof: File exists
mkdir: models: File exists


In [3]:
# ====================================================
# Configurations
# ====================================================
class CFG:
    DATA_PATH = Path('.')
    OOF_DATA_PATH = Path('./oof')
    MODEL_DATA_PATH = Path('./models')
    METHOD_LIST = ['catboost', 'lightgbm']
    seed = random.randint(0, 100)
    n_folds = 10
    target_col = 'attendance'
    USE_PLAYER_FEATURES = False
    num_boost_round = 50500
    early_stopping_round = 500
    verbose = 2000
    boosting_type = 'dart'
    lgb_params = {
        'objective':'regression',
        'metric': 'rmse',
        'boosting': 'dart',
        'n_jobs': -1,
        'seed': seed,
        'feature_pre_filter': False,
        'lambda_l1': 0.0,
        'lambda_l2': 0.0,
        'num_leaves': 4,
        'feature_fraction': 0.9159999999999999,
        'bagging_fraction': 0.808098964054993,
        'bagging_freq': 1,
        'min_child_samples': 20,
        }
    cat_params = {
        'loss_function': 'RMSE',
        'iterations': num_boost_round,
        'random_seed': seed,
        'thread_count': -1,
        'depth': 5,
        'learning_rate': 0.010809655967120722,
        'random_strength': 83,
        'bagging_temperature': 94.09902179801527,
        'od_type': 'Iter',
        'od_wait': 49
        }

In [4]:
# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(CFG.seed)

In [5]:
# ====================================================
# LightGBM Metric
# ====================================================
def lgb_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'rmse', np.sqrt(mean_squared_error(y_true, y_pred)), False

# ====================================================
# Catboost Metric
# ====================================================
class CatboostMetric(object):
    def get_final_error(self, error, weight): return error
    def is_max_optimal(self): return False
    def evaluate(self, approxes, target, weight):
        error = np.sqrt(mean_squared_error(np.array(target), approxes))
        return error, 0

In [6]:
def lightgbm_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame):
    # Load the encoders from a pickle file
    with open('encoders.pkl', 'rb') as f:
        encoders = pickle.load(f)

    # Apply the encoding map to the relevant columns
    categorical_cols = x_train.select_dtypes(include=['object']).columns.tolist()
    for fold in range(5):  # replace 5 with your number of folds
        for col in categorical_cols:
            x_train.loc[x_train['fold'] == fold, f'{col}_enc'] = x_train.loc[x_train['fold'] == fold, col].map(encoders[f'fold_{fold}_{col}'])
            x_valid.loc[x_valid['fold'] == fold, f'{col}_enc'] = x_valid.loc[x_valid['fold'] == fold, col].map(encoders[f'fold_{fold}_{col}'])

    # Create LightGBM datasets
    train_data = lgb.Dataset(x_train, label=y_train, categorical_feature=categorical_cols)
    valid_data = lgb.Dataset(x_valid, label=y_valid, categorical_feature=categorical_cols)

    # Train model
    model = lgb.train(
        params=CFG.lgb_params,
        train_set=train_data,
        num_boost_round=20000,
        valid_sets=[valid_data],
        callbacks=[lgb.early_stopping(CFG.early_stopping_round, verbose=True), lgb.log_evaluation(CFG.verbose)]
        )

    # Predict validation
    valid_pred = model.predict(x_valid)
    return model, valid_pred

def catboost_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, categorical_features: list):
    # Define model
    model = CatBoostRegressor(**CFG.cat_params, cat_features=categorical_features)

    # Fit model
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], early_stopping_rounds=CFG.early_stopping_round, verbose=CFG.verbose)

    # Predict validation
    valid_pred = model.predict(x_valid)

    return model, valid_pred

def gradient_boosting_model_cv_training(method: str, train_df: pd.DataFrame, features: list, categorical_features: list):
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train_df))
    oof_fold = np.zeros(len(train_df))
    kfold = KFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
    for fold, (train_index, valid_index) in enumerate(kfold.split(train_df, train_df[CFG.target_col])):
        print('-'*50)
        print(f'{method} training fold {fold + 1}')
        x_train = train_df[features].iloc[train_index]
        y_train = train_df[CFG.target_col].iloc[train_index]
        x_valid = train_df[features].iloc[valid_index]
        y_valid = train_df[CFG.target_col].iloc[valid_index]
        if method == 'lightgbm':
            model, valid_pred = lightgbm_training(x_train, y_train, x_valid, y_valid)
        if method == 'catboost':
            model, valid_pred = catboost_training(x_train, y_train, x_valid, y_valid, categorical_features)

        # Save best model
        pickle.dump(model, open(CFG.MODEL_DATA_PATH / f'{method}_fold{fold + 1}_seed{CFG.seed}_ver{CFG.boosting_type}.pkl', 'wb'))
        # Save encoders only for lightgbm and xgboost
        # if method in ['lightgbm', 'xgboost']:
        #     pickle.dump(encoders[fold], open(CFG.MODEL_DATA_PATH / f'{method}_encoders_fold{fold + 1}_seed{CFG.seed}_ver{CFG.boosting_type}.pkl', 'wb'))

        # Add to out of folds array
        oof_predictions[valid_index] = valid_pred
        oof_fold[valid_index] = fold + 1
        del x_train, x_valid, y_train, y_valid, model, valid_pred
        gc.collect()

    # Compute out of folds metric
    score = np.sqrt(mean_squared_error(train_df[CFG.target_col], oof_predictions))
    print(f'{method} our out of folds CV score is {score}')
    # Create a dataframe to store out of folds predictions
    # oof_df = pd.DataFrame({'id': train_df['id'], CFG.target_col: train_df[CFG.target_col], f'{method}_prediction': oof_predictions, 'fold': oof_fold})
    # oof_df.to_csv(CFG.MODEL_DATA_PATH / f'oof_{method}_seed{CFG.seed}_ver{CFG.boosting_type}.csv', index = False)

In [7]:
train_df = pd.read_csv(CFG.DATA_PATH / 'train.csv')
venue_info_df = pd.read_csv(CFG.DATA_PATH / 'venue_information.csv')
test_df = pd.read_csv(CFG.DATA_PATH / 'test.csv')
test_df[CFG.target_col] = -1
match_reports_df = pd.read_csv('match_reports.csv')
holidays_in_japan_df = pd.read_csv('holidays_in_japan.csv')
submission_df = pd.read_csv(CFG.DATA_PATH / 'sample_submit.csv')
all_df = pd.concat([train_df, test_df])

In [8]:
# match_reports_df を 'id' カラムで all_df と結合します
all_df = pd.merge(all_df, match_reports_df, on='id', how='left')

# venue_info_df を 'venue' カラムで all_df と結合します
all_df = pd.merge(all_df, venue_info_df, on='venue', how='left')

# holidays_in_japan_df を 'match_date' カラムで all_df と結合します
all_df['match_date'] = pd.to_datetime(all_df['match_date'])
holidays_in_japan_df['holiday_date'] = pd.to_datetime(holidays_in_japan_df['holiday_date'])
all_df['match_date'] = all_df['match_date'].dt.date
holidays_in_japan_df['holiday_date'] = holidays_in_japan_df['holiday_date'].dt.date

# もう一度 datetime 型に戻します
all_df['match_date'] = pd.to_datetime(all_df['match_date'])
holidays_in_japan_df['holiday_date'] = pd.to_datetime(holidays_in_japan_df['holiday_date'])

all_df = pd.merge(all_df, holidays_in_japan_df, left_on='match_date', right_on='holiday_date', how='left')

In [9]:
import feature_engineering as fe

all_df = fe.apply_feature_engineering(all_df)
all_df = fe.process_periodic_features(all_df)
all_df = fe.add_geographical_features(all_df, venue_info_df)
all_df = fe.add_grouped_statistics(all_df)
all_df = all_df.dropna()
all_df = fe.HDBSCAN_featuring(all_df)
all_df = all_df.drop(['venue', 'address', 'description', 'match_date', 'kick_off_time'], axis=1)

# 最後に、訓練データとテストデータに再度分割します
train_df = all_df[all_df['attendance'] != -1].dropna()
test_df = all_df[all_df['attendance'] == -1]

all_df = fe.compute_knn_features_and_preprocess(train_df, test_df, CFG.target_col, k=3, folds=5)

# 最後に、訓練データとテストデータに再度分割します
train_df = all_df[all_df['attendance'] != -1].dropna()
test_df = all_df[all_df['attendance'] == -1]

In [10]:
pd.set_option('display.max_rows', None)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 257 entries, 0 to 256
Columns: 133 entries, id to knn_avg_dist_3
dtypes: bool(4), float64(73), int32(5), int64(46), object(5)
memory usage: 257.0+ KB


In [11]:
missing_rows = all_df[all_df.isnull().any(axis=1)]
print(missing_rows)

Empty DataFrame
Columns: [id, section, round, home_team, away_team, weather, temperature, humidity, attendance, home_team_score, away_team_score, capacity, スカパー!, スカパー!プレミアムサービス, スカパー, e2, スカパー(スカチャンHD、スカチャン), DAZN, スカパー!(パーフェクト チョイス), スカパー光, e2スカチャン, J SPORTS(録), NHK BS1, e2スカチャンHD, BS, e2(スカチャン!), テレ玉, BS-i, BS-TBS, e2(スカチャン!HV), 静岡放送, NHK総合, holiday_flag, long_weekend_flag, discomfort_index, home_team_rank, away_team_rank, home_team_last_year_rank, away_team_last_year_rank, rank_diff, rank_diff_abs, last_year_rank_diff, last_year_rank_diff_abs, diff_score, home_team_avg_conceded_last_3, away_team_avg_conceded_last_3, home_team_scored, away_team_scored, home_team_conceded, away_team_conceded, home_team_avg_points_last_3, home_team_avg_points_last_5, away_team_avg_points_last_3, away_team_avg_points_last_5, home_team_avg_scored_last_3, home_team_avg_scored_last_5, away_team_avg_scored_last_3, away_team_avg_scored_last_5, home_team_winning_streak, away_team_winning_streak, home_team_lo

In [12]:
def calc_smooth_mean(df, by, on, m):
    # Compute the global mean
    mean = df[on].mean()

    # Compute the number of values and the mean of each group
    agg = df.groupby(by)[on].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + m * mean) / (counts + m)

    # Replace each value by the according smoothed mean
    return df[by].map(smooth)

categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
kf = KFold(n_splits=10, shuffle=True, random_state=CFG.seed)
encoders = {}  # to save the encoding map for each fold

train_df['fold'] = -1  # Create a new column to store the fold number

for fold, (train_idx, valid_idx) in enumerate(kf.split(train_df)):
    train_data = train_df.loc[train_idx]
    valid_data = train_df.loc[valid_idx]

    valid_data['fold'] = fold  # Assign the fold number to the valid_data fold column

    for col in categorical_cols:
        valid_data[f'{col}_enc'] = calc_smooth_mean(train_data, by=col, on=CFG.target_col, m=300)
        encoders[f'fold_{fold}_{col}'] = valid_data[f'{col}_enc'].to_dict()  # save the encoding map

    train_df.loc[valid_idx] = valid_data

# Save the encoders to a pickle file
with open('encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f)

In [13]:
import model_tuner
train_df = model_tuner.select_features_and_tune_parameters(train_df, 'attendance', CFG.seed)

Categorical columns in X: ['home_team', 'away_team', 'venue_prefecture', 'venue_region', 'away_prefecture']


ValueError: could not convert string to float: '大分'

In [ ]:
# 'Id'や'Target'といった特定のカラムを除外した全てのカラムを特徴量とする場合
features = train_df.columns.drop(['id', 'attendance'])

# または、データ型が 'object'（文字列）または 'category' のカラムをカテゴリカル特徴量とする場合
categorical_features = train_df.select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
train_df.info(verbose=True)

In [ ]:
# パラメータチューニング
CFG.cat_params = model_tuner.tune_model(train_df[features], train_df[CFG.target_col], n_trials=30)

In [ ]:
for method in CFG.METHOD_LIST:
    gradient_boosting_model_cv_training(method, train_df, features, categorical_features)

In [ ]:
def lightgbm_inference(x_test: pd.DataFrame):
    # Load the encoders from a pickle file
    with open('encoders.pkl', 'rb') as f:
        encoders = pickle.load(f)

    # Apply the encoding map to the relevant columns
    categorical_cols = x_test.select_dtypes(include=['object', 'category']).columns
    for fold in range(CFG.n_folds):
        for col in categorical_cols:
            x_test[f'{col}_enc'] = x_test[col].map(encoders[f'fold_{fold}_{col}'])

    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(CFG.MODEL_DATA_PATH / f'lightgbm_fold{fold + 1}_seed{CFG.seed}_ver{CFG.boosting_type}.pkl', 'rb'))
        categorical_cols = x_test.select_dtypes(include=['object', 'category']).columns
        for col in categorical_cols:
            x_test[col] = x_test[col].astype('category')
        test_pred += model.predict(x_test)
    return test_pred / CFG.n_folds

def catboost_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(CFG.MODEL_DATA_PATH / f'catboost_fold{fold + 1}_seed{CFG.seed}_ver{CFG.boosting_type}.pkl', 'rb'))
        test_pred += model.predict(x_test)
    return test_pred / CFG.n_folds

def gradient_boosting_model_inference(method: str, test_df: pd.DataFrame, features: list):
    x_test = test_df[features]
    if method == 'lightgbm':
        test_pred = lightgbm_inference(x_test)
    if method == 'catboost':
        test_pred = catboost_inference(x_test)
    return test_pred


for method in CFG.METHOD_LIST:
    test_df[f'{method}_pred'] = gradient_boosting_model_inference(method, test_df, features)


# アンサンブルの結果を保存
test_df['final_pred'] = 0.5 * test_df['lightgbm_pred'] + 0.5 * test_df['catboost_pred']
test_df[['id','final_pred']].to_csv(f'submissions/EMS_{CFG.n_folds}folds_submission_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}_{CFG.seed}.csv', index=False, header=False)

# LGBのみの結果を保存
test_df['final_pred'] = 1.0 * test_df['lightgbm_pred'] + 0.0 * test_df['catboost_pred']
test_df[['id','final_pred']].to_csv(f'submissions/OnlyLGB_{CFG.n_folds}folds_submission_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}_{CFG.seed}.csv', index=False, header=False)

# Catのみの結果を保存
test_df['final_pred'] = 0.0 * test_df['lightgbm_pred'] + 1.0 * test_df['catboost_pred']
test_df[['id','final_pred']].to_csv(f'submissions/OnlyCat_{CFG.n_folds}folds_submission_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}_{CFG.seed}.csv', index=False, header=False)